Why some batches are spoiling ealier than others.
One of the mixed four chemicals causes the food to spoil.

In [1]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DogFoodChemicals').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 08:20:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [6]:
df.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

In [8]:
df.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=[
    'A', 'B', 'C', 'D'
], outputCol='features')

In [10]:
output = assembler.transform(df)

In [11]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
output.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0, features=DenseVector([4.0, 2.0, 12.0, 3.0]))]

In [13]:
final_df = output.select('features', 'Spoiled')

In [14]:
final_df.head(1)

[Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [15]:
train_df, test_df = final_df.randomSplit([0.7, 0.3])

In [16]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [17]:
dtc = DecisionTreeClassifier(labelCol='Spoiled', featuresCol='features')
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')
gbt = GBTClassifier(labelCol='Spoiled', featuresCol='features')

In [18]:
dtc_model = dtc.fit(train_df)
rfc_model = rfc.fit(train_df)
gbt_model = gbt.fit(train_df)

In [32]:
dtc_model.featureImportances

SparseVector(4, {0: 0.0176, 1: 0.0173, 2: 0.9641, 3: 0.001})

In [33]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0238, 1: 0.0338, 2: 0.9182, 3: 0.0242})

In [34]:
gbt_model.featureImportances

SparseVector(4, {0: 0.028, 1: 0.0502, 2: 0.9002, 3: 0.0216})

In [19]:
dtc_preds = dtc_model.transform(test_df)
rfc_preds = rfc_model.transform(test_df)
gbt_preds = gbt_model.transform(test_df)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
binary_eval = BinaryClassificationEvaluator(labelCol='Spoiled')

In [22]:
print('DTC')
print(binary_eval.evaluate(dtc_preds))

DTC
0.8644649621212122


In [23]:
print('RFC')
print(binary_eval.evaluate(rfc_preds))

RFC
0.9848484848484848


In [24]:
print('GBT')
print(binary_eval.evaluate(gbt_preds))

GBT
22/09/15 08:35:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/09/15 08:35:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
0.9575047348484849


In [26]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [29]:
gbt_preds.show()

+-------------------+-------+--------------------+--------------------+----------+
|           features|Spoiled|       rawPrediction|         probability|prediction|
+-------------------+-------+--------------------+--------------------+----------+
| [1.0,1.0,10.0,8.0]|    1.0|[1.54350200272498...|[0.95635347857270...|       0.0|
| [1.0,1.0,12.0,4.0]|    1.0|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  [1.0,2.0,9.0,1.0]|    0.0|[1.31653715721666...|[0.93296009026163...|       0.0|
|[1.0,4.0,13.0,10.0]|    1.0|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  [1.0,5.0,8.0,5.0]|    0.0|[1.54393867073898...|[0.95638991840537...|       0.0|
|[1.0,5.0,12.0,10.0]|    1.0|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  [1.0,6.0,8.0,1.0]|    0.0|[1.54399407464195...|[0.95639453977048...|       0.0|
|[1.0,6.0,11.0,10.0]|    1.0|[-1.3477485174282...|[0.06323958920405...|       1.0|
| [1.0,7.0,11.0,9.0]|    1.0|[-1.5795745216940...|[0.04073229015298...|       1.0|
| [1

I didn't actually need to perform the split data op. The indicator is the featureImportances method ran on rfc_model. 
This on indicates that the "C" column parameter is the reason behind the Spoiledge. 